**Тестовое задание на вакансию AI-разработчик junior (Инлайн), выполнил Имамутдинов Артур**

**Задание:**

Создайте модель, обрабатывающую фрагмент текста и определяющую
какой вид продукции в нём содержится.

Выполненное задание пришлите ссылкой на Google Collab.
Датасет поделите на 80% / 20% - обучающая/тестовая выборки.

Используйте библиотеку PyTorch

Датасет: https://axe.inline-ltd.ru/data/meatinfo.csv

Виды продукции (брать только виды продукции, для которых в датасете есть не менее 500 примеров):

Баранина
Ягнятина
Индейка
Говядина
Свинина
Кура
Цыплено
Гусь
Буйволятина
Оленина
Конина
Телятина
Кролик
Утка
Куропатка
Перепел
Глухарь
Страус
Заяц
Кенгуру
Изюбр
Кабан
Коза
Косуля
Лось
Марал
Медвежатина
Бобер
Цесарка
Нутрия
Рябчик
Тетерев
Фазан
Як


Примеры входных текстов и видов продукции:
Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг. -> Свинина

Мясо премиум Предлагаем котлетное мясо мраморной говядины. -> Говядина

спинка цб -> Цыпленок

Проверьте вашу модель на образцах
Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)
Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.

Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:

Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.


In [ ]:
import pandas as pd # для работы с таблицей
from pymystem3 import Mystem # для лемматизации
from string import punctuation # для отброса пунктуации
import re # для регулярных выражений

# русские стоп слова
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Процесс выгрузки и обработки данных

In [ ]:
# https://axe.inline-ltd.ru/data/meatinfo.csv
df = pd.read_csv('/content/meatinfo.csv',sep = ';')

In [ ]:
df.head(10)

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
5,"Баранина, 12 частей, охл. цена 230 руб.",Баранина
6,"Баранина, 12 частей, охл. цена 270 руб.",Баранина
7,"Баранина, 12 частей, охл. цена 280 руб.",Баранина
8,"Баранина, 12 частей, охл. цена 285 руб.",Баранина
9,"Баранина, 12 частей, охл. цена 310 руб.",Баранина


In [ ]:
df.shape

(17893, 2)

In [ ]:
df.isnull().sum()

,0
text,0
mtype,1


In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
df.shape

(17892, 2)

Токенизация текста

In [ ]:
mystem = Mystem()
russian_stopwords = stopwords.words('russian')

In [ ]:
def preprocess_text(text):
  tokens = re.sub(r'[\d]','',re.sub(r'\([\w\s]*\)', '',text))
  tokens = mystem.lemmatize(tokens.lower())
  tokens = [token for token in tokens if token not in russian_stopwords and token != ' ' and token.strip() not in punctuation]
  return ' '.join(tokens)

Пример обработки текста

In [ ]:
print(preprocess_text(df['text'][21]))
print(df['text'][21])
print(df['mtype'][21])

баранина замораживать часть баранинный дагестанский разделывать часть замораживать упаковывать гофротару иметься весь документ доставка москва бесплатно цена руб
баранина (ягнята) замороженная 6 частей баранинна (ягнята) дагестанская, разделанные на 6 частей замороженные (шоковая заморозка) упакованная в гофротару, имеются все документы.
 доставка до Москвы бесплатно. цена 265-285 руб.
Баранина


In [ ]:
for i in range(df.shape[0]):
  df.loc[i,'text'] = preprocess_text(df['text'][i])

In [ ]:
df.head(10)

,text,mtype
0,часть баранина часть баранина,Баранина
1,баранина часть зам цена руб,Баранина
2,баранина часть зам цена руб,Баранина
3,баранина часть охла,Баранина
4,баранина часть охла цена руб,Баранина
5,баранина часть охла цена руб,Баранина
6,баранина часть охла цена руб,Баранина
7,баранина часть охла цена руб,Баранина
8,баранина часть охла цена руб,Баранина
9,баранина часть охла цена руб,Баранина


Определение классов (меток), записей которых >500 шт

In [ ]:
mtypes = df['mtype'].unique()
print(mtypes)

['Баранина' 'Ягнятина' 'Индейка' 'Говядина' 'Свинина' 'Кура' 'Цыпленок'
 'Гусь' 'Буйволятина' 'Оленина' 'Конина' 'Телятина' '125р.' 'Кролик'
 'Утка' 'Куропатка' 'Парагвай'
 'Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг'
 'Перепел' 'Глухарь' 'Страус' 'Заяц' 'Кенгуру' 'Изюбр' 'Кабан'
 '295,00 руб|кг' 'Коза' 'Косуля' ' Лопаточная часть (Chuck) буйвол '
 'Лось' 'Марал' 'Медвежатина' 'Бобер' 'Цесарка' 'Нутрия' 'Feb-20' 'Mar-20'
 '(OFFAL EXP №4407 Аргентина)' 'OFFAL EXP №4407 Аргентина' 'индейка'
 'свиниеа' 'утка' 'цыпленок' 'свинина' 'Рябчик' 'Тетерев' 'говядина'
 'Фазан' 'Як']


In [ ]:
classes = []
for mtype in mtypes:
  if df.loc[df['mtype'].isin([mtype])].shape[0]<500:
    print('Для-',mtype,'-записей нашлось',df.loc[df['mtype'].isin([mtype])].shape[0])
  else:
    classes.append(mtype)

Для- Ягнятина -записей нашлось 76
Для- Гусь -записей нашлось 125
Для- Буйволятина -записей нашлось 75
Для- Оленина -записей нашлось 193
Для- Конина -записей нашлось 176
Для- Телятина -записей нашлось 98
Для- 125р. -записей нашлось 1
Для- Кролик -записей нашлось 334
Для- Утка -записей нашлось 195
Для- Куропатка -записей нашлось 7
Для- Парагвай -записей нашлось 2
Для- Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг -записей нашлось 1
Для- Перепел -записей нашлось 54
Для- Глухарь -записей нашлось 1
Для- Страус -записей нашлось 10
Для- Заяц -записей нашлось 2
Для- Кенгуру -записей нашлось 2
Для- Изюбр -записей нашлось 3
Для- Кабан -записей нашлось 24
Для- 295,00 руб|кг -записей нашлось 1
Для- Коза -записей нашлось 1
Для- Косуля -записей нашлось 8
Для-  Лопаточная часть (Chuck) буйвол  -записей нашлось 1
Для- Лось -записей нашлось 20
Для- Марал -записей нашлось 7
Для- Медвежатина -записей нашлось 4
Для- Бобер -записей нашлось 1
Для- Цесарка -записей н

In [ ]:
classes

['Баранина', 'Индейка', 'Говядина', 'Свинина', 'Кура', 'Цыпленок']

Выбрасываем записи, меток которых <500

In [ ]:
for mtype in mtypes:
  if mtype not in classes:
    df.drop(df[df['mtype'].isin([mtype])].index.to_list(),axis=0,inplace = True)

In [ ]:
df.shape

(16438, 2)

In [ ]:
for mtype in mtypes:
  print('Для-',mtype,'-записей нашлось',df.loc[df['mtype'].isin([mtype])].shape[0])

Для- Баранина -записей нашлось 1116
Для- Ягнятина -записей нашлось 0
Для- Индейка -записей нашлось 1337
Для- Говядина -записей нашлось 8422
Для- Свинина -записей нашлось 3050
Для- Кура -записей нашлось 1571
Для- Цыпленок -записей нашлось 942
Для- Гусь -записей нашлось 0
Для- Буйволятина -записей нашлось 0
Для- Оленина -записей нашлось 0
Для- Конина -записей нашлось 0
Для- Телятина -записей нашлось 0
Для- 125р. -записей нашлось 0
Для- Кролик -записей нашлось 0
Для- Утка -записей нашлось 0
Для- Куропатка -записей нашлось 0
Для- Парагвай -записей нашлось 0
Для- Говядина, полутуши, 1 категория,  охл., Россия, подвес, В наличии, 10 тонн, 270 руб. кг -записей нашлось 0
Для- Перепел -записей нашлось 0
Для- Глухарь -записей нашлось 0
Для- Страус -записей нашлось 0
Для- Заяц -записей нашлось 0
Для- Кенгуру -записей нашлось 0
Для- Изюбр -записей нашлось 0
Для- Кабан -записей нашлось 0
Для- 295,00 руб|кг -записей нашлось 0
Для- Коза -записей нашлось 0
Для- Косуля -записей нашлось 0
Для-  Лопаточн

Замена категориальных значений в наборе данных

['Баранина', 'Индейка', 'Говядина', 'Свинина', 'Кура', 'Цыпленок']


In [ ]:
dictionary_classes = {'mtype' : {'Баранина' : 0, 'Индейка' : 1, 'Говядина' : 2, 'Свинина' : 3, 'Кура' : 4, 'Цыпленок' : 5}}
df = df.replace(dictionary_classes)
df = df.reset_index(drop=True)

<ipython-input-19-52806126241b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(dictionary_classes)


In [ ]:
df

,text,mtype
0,часть баранина часть баранина,0
1,баранина часть зам цена руб,0
2,баранина часть зам цена руб,0
3,баранина часть охла,0
4,баранина часть охла цена руб,0
...,...,...
16433,цыпленок четвертина задний,5
16434,цыпленок четвертина задний цена руб,5
16435,цб шея п ф свеженка гост зам пак шея куриный п...,5
16436,цыпленок шея кожа,5


p.s можно лучше обработать датасет, включив в него записи, у которых вид продукции отличается от классов незначительно (например вид продукции написанн с ошибкой)

## Создание словаря и преобразование текста в тензор по словарю

In [ ]:
# torchtext (0.18) последней версии работает с версией torch 2.3.0, с новыми почему то конфликтует https://pypi.org/project/torchtext/
!pip install -U torch==2.3.0
!pip install -U torchtext==0.18

In [ ]:
import torch # для тензоров и работы с ними, также для машинного обучения
import torchtext # для работы с словарем
torchtext.disable_torchtext_deprecation_warning()
import torchtext.vocab.vocab

In [ ]:
def create_tokens(dataseries):
  tokens = []
  for data in dataseries:
    words = mystem.lemmatize(data)
    for word in words:
      if word != ' ' and word not in russian_stopwords and word.strip() not in punctuation:
        tokens.append(word)
  return tokens

In [ ]:
def create_dict(spisok):
  count = 0
  dictionary = {}
  for word in spisok:
    if word not in dictionary:
      dictionary[word] = count
      count += 1
  return dictionary

In [ ]:
tokens = create_tokens(df['text'])
dictionary = create_dict(tokens)
vocab = torchtext.vocab.vocab(dictionary)
vocab.set_default_index(-1)

In [ ]:
print(len(tokens))
print(len(dictionary))
print(len(vocab))

238672
8698
8697


In [ ]:
vocab.get_stoi()

{"  '' ": 8694,
 'вполне': 8690,
 ' *****,      \n': 8689,
 '  - ,% ;  ': 8684,
 'тонкость': 8683,
 'ржевский': 8680,
 'серия': 8679,
 'балкария': 8676,
 'кабардино': 8675,
 '  (+) ': 8674,
 'донецк': 8667,
 'киль+': 8666,
 '  , , , ,   ': 8662,
 'радуга': 8661,
 'чесночок': 8655,
 'вишневый': 8653,
 'откалибровать': 8652,
 '  ~+,  ': 8649,
 'машинка': 8647,
 'коптильный': 8642,
 'переносить': 8641,
 ' : -, -, -, -, -, +\n': 8639,
 'пятигорск': 8638,
 'осмал': 8636,
 'означать': 8634,
 '  »),  ': 8633,
 'немой': 8630,
 ' , /, /  ': 8626,
 'витрина': 8625,
 'пригодный': 8623,
 'замечать': 8621,
 'римминг': 8620,
 'свиния': 8617,
 'конкретно': 8616,
 'обго': 8615,
 'свиниг': 8614,
 'накопительный': 8607,
 ' ,)   ': 8606,
 'тdреrsiк': 8590,
 'балахня': 8601,
 'безотходный': 8589,
 'фикалии': 8586,
 '  +, +  ': 8584,
 'ширина': 8583,
 'raw': 8581,
 '  +;\n': 8580,
 'осмаленый': 8573,
 '    -+ -   ': 8570,
 'по-белорусски': 8569,
 'варение': 8568,
 'заветер': 8562,
 'госг': 8561,
 'свмочево

Пример ембеддинга

In [ ]:
print(df['text'][50].split())
print([vocab[token] for token in df['text'][50].split()])

['баранина', 'отруб', 'зам', 'цена', 'руб']
[0, 39, 1, 2, 3]


Создание тензоров для обучения по ембеддингу

In [ ]:
# length = len(vocab)
def create_tensors(data,length):
  X_tensor = torch.zeros(data.shape[0],length)
  y_tensor = torch.from_numpy(data['mtype'].values)
  for i,text in enumerate(data['text']):
    indexes = [vocab[token] for token in text.split()]
    for index in indexes:
      X_tensor[i][index] = 1
  return X_tensor,y_tensor

In [ ]:
X_tensor,y_tensor = create_tensors(df,len(vocab))

## Создание модели


In [ ]:
import torch.nn as nn # для скрытых слоев
from sklearn.model_selection import train_test_split # для разделения датасета на выборки
import numpy as np # для математики
import random # фиксирование рандомных сидов

p.s можно улучшить архитектуру сети, добавить больше скрытых слоев, функции активации, использование батч-нормализации, дропауты. Но в данном примере улучшения будут незначительны


In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_classes,classes):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.classes = classes

    def forward(self, x):
        output = self.fc1(x)
        logits = self.fc2(output)
        return logits

    def predict(self,x):
      example = [vocab[token] for token in preprocess_text(x).split()]
      example_tensor = torch.zeros(len(vocab))
      for index in example:
        example_tensor[index] = 1
      return self.classes[self.forward(example_tensor).argmax().item()]

p.s Можно изменить параметры обучения и найти лучшие из них, добавить early stop (от переобучения), подбор параметров в optimizer, количество батчей, learning rate.

In [ ]:
# Параметры обучения
vocab_size = len(vocab)
hidden_dim = 128
num_classes = len(classes)
batch_size = 32
num_epochs = 3
learning_rate = 0.001

# Создание модели
model = TextClassifier(vocab_size, hidden_dim, num_classes,classes)

# Определение функции ошибки и оптимайзера
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

80% обучающей выборки, 20% для тестовой выборки

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_tensor,y_tensor,test_size = 0.2,random_state = 15)

In [ ]:
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

## Обучение модели

In [ ]:
# Фиксация рандомных сидов
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

# Определение девайса, на котором будут проходить вычисления
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Сохранение промежуточных результатов
test_accuracy_history = []
test_loss_history = []
train_loss_history = []

X_test = X_test.to(device)
y_test = y_test.to(device)

# Обучение
for epoch in range(num_epochs):
  order = np.random.permutation(len(X_train))
  train_loss_epoch = []
  for start_index in range(0, len(X_train), batch_size):
    model.train()
    optimizer.zero_grad()

    batch_indexes = order[start_index:start_index+batch_size]

    X_batch = X_train[batch_indexes].to(device)
    y_batch = y_train[batch_indexes].to(device)

    preds = model.forward(X_batch)

    loss_value = loss(preds, y_batch)
    train_loss_epoch.append(loss_value.detach().numpy())
    loss_value.backward()
    optimizer.step()

  model.eval()

  train_loss = np.mean(train_loss_epoch)
  train_loss_history.append(train_loss)

  # need to use with torch.no_grad()
  with torch.no_grad():
    test_preds = model.forward(X_test)
    loss_val = loss(test_preds, y_test).data.cpu().item()
    test_loss_history.append(loss_val)

    accuracy = (test_preds.argmax(dim=1)==y_test).float().mean().data.cpu().item()
    test_accuracy_history.append(accuracy)
  print('current train_loss on epoch', train_loss,'current loss_val',loss_val,'accuracy',accuracy)

current train_loss on epoch 0.40959588 current loss_val 0.1560184508562088 accuracy 0.9586374759674072
current train_loss on epoch 0.09950604 current loss_val 0.1595253348350525 accuracy 0.9565085172653198
current train_loss on epoch 0.07357589 current loss_val 0.16676919162273407 accuracy 0.9546837210655212


## Проверяем модель

In [ ]:
examples = ['Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.',
            'Мясо премиум Предлагаем котлетное мясо мраморной говядины.',
            'спинка цб',
            'Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.',
            'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:',
            'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.']

In [ ]:
for example in examples:
  print('For:',example,'prediction is:',model.predict(example))

For: Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг. prediction is: Свинина
For: Мясо премиум Предлагаем котлетное мясо мраморной говядины. prediction is: Говядина
For: спинка цб prediction is: Цыпленок
For: Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону. prediction is: Говядина
For: Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел: prediction is: Кура
For: Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем. prediction is: Говядина
